In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import time
from itertools import combinations
from collections import defaultdict

import numpy as np
from random import sample
from scipy.special import binom

import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns

RNG_SEED = int(time.time())
print("Seed: %s" % RNG_SEED)
overwrite_models = True

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

# train_df.info()
# train_df.head()
# test_df.info()

outcome = train_df["Survived"]
# feature_df = train_df.drop(["Cabin", "Survived", "PassengerId", "Name", "Ticket"], axis=1)
feature_df = train_df.drop(["Survived", "PassengerId", "Cabin", "Name"], axis=1)
feature_df.fillna({"Age": train_df["Age"].median()}, inplace=True)
feature_df.fillna({"Embarked": train_df["Embarked"].mode()[0]}, inplace=True)
feature_df["FamilySize"] = feature_df["SibSp"] + feature_df["Parch"]
feature_df.info()

# test_df = test_df.drop(["Cabin", "PassengerId", "Name", "Ticket"], axis=1)
passenger_ids = test_df["PassengerId"]
test_df = test_df.drop(["PassengerId", "Cabin", "Name"], axis=1)
test_df.fillna({"Age": test_df["Age"].median()}, inplace=True)
test_df.fillna({"Fare": test_df["Fare"].median()}, inplace=True)
test_df["FamilySize"] = test_df["SibSp"] + test_df["Parch"]
test_df.info()

categorical_vars_indices = np.where((feature_df.dtypes == object) | (feature_df.dtypes == bool))[0]
categorical_vars = feature_df.columns[categorical_vars_indices]
le = LabelEncoder()

for var in categorical_vars:
    data = pd.concat([feature_df.loc[:, var], test_df.loc[:, var]])
    le = le.fit(data)
    feature_df.loc[:, var] = le.transform(feature_df.loc[:, var])
    test_df.loc[:, var] = le.transform(test_df.loc[:, var])
    
scaler = MinMaxScaler()
feature_df[:] = scaler.fit_transform(feature_df)
test_df[:] = scaler.fit_transform(test_df)


In [ ]:
skf_cv = StratifiedKFold(5)
# predetermined folds to be used for model stacking
stack_folds = list(StratifiedKFold(5).split(feature_df, outcome))

In [ ]:
def best_accuracy_threshold(proba_preds, truth, cuts=np.arange(0, 1, 0.01)):
    acc = np.full(len(cuts), None)
    for i, cut in enumerate(cuts):
        cv_preds = proba_preds > cut
        acc[i] = accuracy_score(truth, cv_preds)
    best_index = np.argmax(acc)
    fig, ax = plt.subplots()
    ax.plot(cuts, acc)
    ax.set_title("best proba cut = %s" % acc[best_index])
    plt.show()
    return(cuts[best_index], acc[best_index])

def feature_combinations(features, n_features, n_comb):
    for i in range(n_comb):
        yield sample(features, n_features)

def subset_var_selection(model, param_grid, X, y, n_feature_list=None, 
                         n_feature_comb=10):
    features = list(X.columns)
    if n_feature_list is None:
        n_feature_list = range(2, len(features) + 1)
    
    best_tuple = None
    var_scores = defaultdict(list)
    for n_features in n_feature_list:
        
        if type(n_feature_comb) is float:
            # proportion of the total number of feature combinations
            n_comb = n_feature_comb * binom(len(features), n_features)
        elif type(n_feature_comb) is int:
            n_comb = min(n_feature_comb, int(binom(len(features), n_features)))
        else:
            raise TypeError("n_feature_comb must be a float or an int")
            
        for comb in feature_combinations(features, n_features, n_comb):
            # print(comb)
            gs = GridSearchCV(model, param_grid, cv=skf_cv, scoring="neg_log_loss")
            cv_opt = gs.fit(X.loc[:, comb], y)
            # print(comb, cv_opt.best_score_)
            if best_tuple is None or cv_opt.best_score_ > best_tuple[2]:
                best_tuple = (comb, cv_opt.best_params_, cv_opt.best_score_)
            
            for var in comb:
                var_scores[var].append(cv_opt.best_score_)
                
    var_avg_score = []
    for var, scores in var_scores.items():
        var_avg_score.append((var, np.mean(scores)))
                
    return best_tuple, var_avg_score
    

In [ ]:
# variable selection + parameter tuning with cross-validation
"""
nb_param_grid = {"alpha": np.arange(1e-10, 2, 0.1)}
best_tuple, var_imp = subset_var_selection(MultinomialNB(), nb_param_grid, 
                                           feature_df, outcome, n_feature_comb=50)
nb_best_vars, nb_best_params, nb_best_score = best_tuple
print(var_imp)
"""

# from a previous run of the code above, for the submission
nb_best_vars = ['Pclass', 'Parch', 'Embarked', 'Fare', 'Ticket', 'FamilySize', 'Age', 'Sex']
nb_best_params = {'alpha': 1e-10}
nb_best_score = -0.590673933422132

best_nb = MultinomialNB(alpha=nb_best_params["alpha"])
print(nb_best_vars)

# Use the fold predictions to get the best cutoff on the 
# class probabilities. This probably introduces a slight 
# risk of overfitting since the cutoff is not computed on a per-fold 
# basis.
# The (hard?) fold predictions will be used for model stacking
nb_cv_proba_preds = cross_val_predict(best_nb, feature_df.loc[:, nb_best_vars], outcome,
                                      cv=stack_folds, method="predict_proba")
nb_cv_proba_preds = np.array([elt[1] for elt in nb_cv_proba_preds])
nb_best_cut, nb_best_acc = best_accuracy_threshold(nb_cv_proba_preds, outcome)
nb_cv_preds = nb_cv_proba_preds > nb_best_cut
print(accuracy_score(outcome, nb_cv_preds))

# the full model is used to predict the test set
full_nb_model = best_nb.fit(feature_df.loc[:, nb_best_vars], outcome)
nb_test_proba_preds = full_nb_model.predict_proba(test_df.loc[:, nb_best_vars])
nb_test_proba_preds = np.array([elt[1] for elt in nb_test_proba_preds])
nb_test_preds = nb_test_proba_preds > nb_best_cut

In [ ]:
"""
# parameter tuning
xgb_reg_ratios = [0.1, 0.5, 0.9, 0.99]
xgb = XGBClassifier(random_state=RNG_SEED)
xgb_param_grid = {"n_estimators": [200, 500],
                  "max_depth": [1, 2, 3],
                  "learning_rate": [0.01, 0.05, 0.1, 0.2],
                  "reg_lambda": xgb_reg_ratios,
                  "reg_alpha": xgb_reg_ratios}
gs_xgb = GridSearchCV(xgb, xgb_param_grid, cv=skf_cv, 
                      scoring="neg_log_loss")
cv_opt_xgb = gs_xgb.fit(feature_df, outcome)
cv_opt_xgb_model = cv_opt_xgb.best_estimator_

print(cv_opt_xgb_model)

# feature importances
fig, ax = plt.subplots()
ax.bar(range(len(cv_opt_xgb_model.feature_importances_)), 
        cv_opt_xgb_model.feature_importances_)
ax.set_title(cv_opt_xgb.best_score_)
plt.show()
"""

# from a previous run of the code above, for the submission
cv_opt_xgb_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
                       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
                       n_jobs=1, nthread=None, objective='binary:logistic',
                       random_state=RNG_SEED, reg_alpha=0.1, reg_lambda=0.5,
                       scale_pos_weight=1, seed=None, silent=True, subsample=1)

# The fold predictions will be used for model stacking
xgb_cv_preds = cross_val_predict(cv_opt_xgb_model, feature_df, outcome,
                                 cv=stack_folds, method="predict")
print(accuracy_score(outcome, xgb_cv_preds))

# the full model is used to predict the test set
full_xgb_model = cv_opt_xgb_model.fit(feature_df, outcome)
xgb_test_preds = full_xgb_model.predict(test_df)


In [ ]:
# fold predictions of the base learners to be used for 
# training the meta model
# stack_cv_preds = np.vstack([nb_cv_preds, xgb_cv_preds]).T
stack_cv_preds = np.vstack([nb_cv_proba_preds, xgb_cv_preds]).T

# meta model
lr = LogisticRegression(random_state=RNG_SEED, max_iter=10000)

# cross-validation (note: using the same stack folds) to assess 
# the accuracy of the meta model
stack_cv_proba_preds = cross_val_predict(lr, stack_cv_preds, outcome,
                                         cv=stack_folds, method="predict_proba")
stack_cv_proba_preds = np.array([elt[1] for elt in stack_cv_proba_preds])
stack_best_cut, stack_best_acc = best_accuracy_threshold(stack_cv_proba_preds, outcome)
# stack_cv_preds = stack_cv_proba_preds > stack_best_cut


In [ ]:
lr = lr.fit(stack_cv_preds, outcome)
print(lr.coef_)
# stack_test_preds = np.vstack([nb_test_preds, xgb_test_preds]).T
stack_test_preds = np.vstack([nb_test_proba_preds, xgb_test_preds]).T

test_proba_preds = lr.predict_proba(stack_test_preds)
test_proba_preds = [elt[1] for elt in test_proba_preds]
test_preds = test_proba_preds > stack_best_cut
test_preds = test_preds.astype(int)

submission = pd.DataFrame({"PassengerId": passenger_ids, "Survived": test_preds})
submission.to_csv('model_stacking_nb_xgb_to_lr.csv', index=False)
